<a href="https://www.kaggle.com/code/tanakataro73/kaggle-book-morohashi-08-mlb-engagement?scriptVersionId=172598257" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mlb-player-digital-engagement-forecasting/players.csv
/kaggle/input/mlb-player-digital-engagement-forecasting/example_sample_submission.csv
/kaggle/input/mlb-player-digital-engagement-forecasting/teams.csv
/kaggle/input/mlb-player-digital-engagement-forecasting/seasons.csv
/kaggle/input/mlb-player-digital-engagement-forecasting/example_test.csv
/kaggle/input/mlb-player-digital-engagement-forecasting/train_updated.csv
/kaggle/input/mlb-player-digital-engagement-forecasting/train.csv
/kaggle/input/mlb-player-digital-engagement-forecasting/awards.csv
/kaggle/input/mlb-player-digital-engagement-forecasting/mlb/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/mlb-player-digital-engagement-forecasting/mlb/__init__.py


In [2]:
import gc
import pickle
import os
import datetime as dt

import matplotlib.pyplot as plt

import lightgbm as lgb

from sklearn.metrics import mean_absolute_error

import warnings
warnings.simplefilter("ignore")

# https://stackoverflow.com/a/8885688
pd.options.display.float_format = "{:10.4f}".format

In [3]:
train = pd.read_csv("/kaggle/input/mlb-player-digital-engagement-forecasting/train_updated.csv")
print(train.shape)
train.head()

(1308, 12)


,date,nextDayPlayerEngagement,games,rosters,playerBoxScores,teamBoxScores,transactions,standings,awards,events,playerTwitterFollowers,teamTwitterFollowers
0,20180101,"[{""engagementMetricsDate"":""2018-01-02"",""player...",NaN,"[{""playerId"":400121,""gameDate"":""2018-01-01"",""t...",NaN,NaN,"[{""transactionId"":340732,""playerId"":547348,""pl...",NaN,NaN,NaN,"[{""date"":""2018-01-01"",""playerId"":545361,""playe...","[{""date"":""2018-01-01"",""teamId"":147,""teamName"":..."
1,20180102,"[{""engagementMetricsDate"":""2018-01-03"",""player...",NaN,"[{""playerId"":134181,""gameDate"":""2018-01-02"",""t...",NaN,NaN,"[{""transactionId"":339458,""playerId"":621173,""pl...",NaN,NaN,NaN,NaN,NaN
2,20180103,"[{""engagementMetricsDate"":""2018-01-04"",""player...",NaN,"[{""playerId"":425492,""gameDate"":""2018-01-03"",""t...",NaN,NaN,"[{""transactionId"":347527,""playerId"":572389,""pl...",NaN,NaN,NaN,NaN,NaN
3,20180104,"[{""engagementMetricsDate"":""2018-01-05"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-04"",""t...",NaN,NaN,"[{""transactionId"":339549,""playerId"":545343,""pl...",NaN,NaN,NaN,NaN,NaN
4,20180105,"[{""engagementMetricsDate"":""2018-01-06"",""player...",NaN,"[{""playerId"":282332,""gameDate"":""2018-01-05"",""t...",NaN,NaN,"[{""transactionId"":341195,""playerId"":628336,""pl...",NaN,NaN,NaN,NaN,NaN


In [4]:
# 試行錯誤の速度向上のため20200401以降にしぼる
train = train.loc[train["date"] >= 20200401, :].reset_index(drop=True)
print(train.shape)

(487, 12)


In [5]:
def unpack_json(json_str):
    return np.nan if pd.isna(json_str) else pd.read_json(json_str)

def extract_data(
    input_df,
    col="events",
    show=False
):
    output_df = pd.DataFrame()
    
    # なぜrangeでなくarangeを使うのか謎だったが右が参考になる https://stackoverflow.com/a/75533644
    for i in np.arange(len(input_df)):
        if show:
            print("\r{}/{}".format(i+1, len(input_df), end=""))
        try:
            output_df = pd.concat(
                [
                    output_df,
                    unpack_json(input_df[col].iloc[i]),
                ],
                axis=0,
                ignore_index=True,
            )
        except:
            pass
    
    if show:
        print("")
        print(output_df.shape)
        display(output_df.head())

    return output_df

In [6]:
df_engagement = extract_data(train, col="nextDayPlayerEngagement", show=True)

1/487
2/487
3/487
4/487
5/487
6/487
7/487
8/487
9/487
10/487
11/487
12/487
13/487
14/487
15/487
16/487
17/487
18/487
19/487
20/487
21/487
22/487
23/487
24/487
25/487
26/487
27/487
28/487
29/487
30/487
31/487
32/487
33/487
34/487
35/487
36/487
37/487
38/487
39/487
40/487
41/487
42/487
43/487
44/487
45/487
46/487
47/487
48/487
49/487
50/487
51/487
52/487
53/487
54/487
55/487
56/487
57/487
58/487
59/487
60/487
61/487
62/487
63/487
64/487
65/487
66/487
67/487
68/487
69/487
70/487
71/487
72/487
73/487
74/487
75/487
76/487
77/487
78/487
79/487
80/487
81/487
82/487
83/487
84/487
85/487
86/487
87/487
88/487
89/487
90/487
91/487
92/487
93/487
94/487
95/487
96/487
97/487
98/487
99/487
100/487
101/487
102/487
103/487
104/487
105/487
106/487
107/487
108/487
109/487
110/487
111/487
112/487
113/487
114/487
115/487
116/487
117/487
118/487
119/487
120/487
121/487
122/487
123/487
124/487
125/487
126/487
127/487
128/487
129/487
130/487
131/487
132/487
133/487
134/487
135/487
136/487
137/487
138/487
139/

,engagementMetricsDate,playerId,target1,target2,target3,target4
0,2020-04-02,425794,5.1249,9.4340,0.1179,6.1947
1,2020-04-02,571704,0.0389,8.1761,0.0105,2.1304
2,2020-04-02,506702,0.0106,5.0314,0.0082,0.8850
3,2020-04-02,607231,0.0247,2.8302,0.0222,0.5900
4,2020-04-02,543193,0.0071,1.1006,0.0012,0.1967


In [7]:
df_engagement["date_playerId"] = df_engagement["engagementMetricsDate"].str.replace("-", "") + "_" + df_engagement["playerId"].astype(str)
df_engagement.head()

,engagementMetricsDate,playerId,target1,target2,target3,target4,date_playerId
0,2020-04-02,425794,5.1249,9.4340,0.1179,6.1947,20200402_425794
1,2020-04-02,571704,0.0389,8.1761,0.0105,2.1304,20200402_571704
2,2020-04-02,506702,0.0106,5.0314,0.0082,0.8850,20200402_506702
3,2020-04-02,607231,0.0247,2.8302,0.0222,0.5900,20200402_607231
4,2020-04-02,543193,0.0071,1.1006,0.0012,0.1967,20200402_543193


In [8]:
# 推論実施日＝推論対象日の前日
df_engagement["date"] = pd.to_datetime(df_engagement["engagementMetricsDate"], format="%Y-%m-%d") + dt.timedelta(days=-1)
# 推論実施日から曜日と年月の特徴量作成
df_engagement["dayofweek"] = df_engagement["date"].dt.dayofweek
df_engagement["yearmonth"] = df_engagement["date"].astype(str).apply(lambda x: x[:7])
df_engagement.head()

,engagementMetricsDate,playerId,target1,target2,target3,target4,date_playerId,date,dayofweek,yearmonth
0,2020-04-02,425794,5.1249,9.4340,0.1179,6.1947,20200402_425794,2020-04-01,2,2020-04
1,2020-04-02,571704,0.0389,8.1761,0.0105,2.1304,20200402_571704,2020-04-01,2,2020-04
2,2020-04-02,506702,0.0106,5.0314,0.0082,0.8850,20200402_506702,2020-04-01,2,2020-04
3,2020-04-02,607231,0.0247,2.8302,0.0222,0.5900,20200402_607231,2020-04-01,2,2020-04
4,2020-04-02,543193,0.0071,1.1006,0.0012,0.1967,20200402_543193,2020-04-01,2,2020-04


In [9]:
# players.csv
df_players = pd.read_csv("/kaggle/input/mlb-player-digital-engagement-forecasting/players.csv")
print(df_players.shape)
print(df_players["playerId"].agg("nunique"))
df_players.head()

(2061, 12)
2061


,playerId,playerName,DOB,mlbDebutDate,birthCity,birthStateProvince,birthCountry,heightInches,weight,primaryPositionCode,primaryPositionName,playerForTestSetAndFuturePreds
0,665482,Gilberto Celestino,1999-02-13,2021-06-02,Santo Domingo,NaN,Dominican Republic,72,170,8,Outfielder,False
1,593590,Webster Rivas,1990-08-08,2021-05-28,Nagua,NaN,Dominican Republic,73,219,3,First Base,True
2,661269,Vladimir Gutierrez,1995-09-18,2021-05-28,Havana,NaN,Cuba,73,190,1,Pitcher,True
3,669212,Eli Morgan,1996-05-13,2021-05-28,Rancho Palos Verdes,CA,USA,70,190,1,Pitcher,True
4,666201,Alek Manoah,1998-01-09,2021-05-27,Homestead,FL,USA,78,260,1,Pitcher,True


In [10]:
df_players["playerForTestSetAndFuturePreds"] = np.where(df_players["playerForTestSetAndFuturePreds"] == True, 1, 0)
print(df_players["playerForTestSetAndFuturePreds"].sum())
print(df_players["playerForTestSetAndFuturePreds"].mean())

1187
0.5759340126152354


In [11]:
df_engagement.head()

,engagementMetricsDate,playerId,target1,target2,target3,target4,date_playerId,date,dayofweek,yearmonth
0,2020-04-02,425794,5.1249,9.4340,0.1179,6.1947,20200402_425794,2020-04-01,2,2020-04
1,2020-04-02,571704,0.0389,8.1761,0.0105,2.1304,20200402_571704,2020-04-01,2,2020-04
2,2020-04-02,506702,0.0106,5.0314,0.0082,0.8850,20200402_506702,2020-04-01,2,2020-04
3,2020-04-02,607231,0.0247,2.8302,0.0222,0.5900,20200402_607231,2020-04-01,2,2020-04
4,2020-04-02,543193,0.0071,1.1006,0.0012,0.1967,20200402_543193,2020-04-01,2,2020-04


In [12]:
df_players.head()

,playerId,playerName,DOB,mlbDebutDate,birthCity,birthStateProvince,birthCountry,heightInches,weight,primaryPositionCode,primaryPositionName,playerForTestSetAndFuturePreds
0,665482,Gilberto Celestino,1999-02-13,2021-06-02,Santo Domingo,NaN,Dominican Republic,72,170,8,Outfielder,0
1,593590,Webster Rivas,1990-08-08,2021-05-28,Nagua,NaN,Dominican Republic,73,219,3,First Base,1
2,661269,Vladimir Gutierrez,1995-09-18,2021-05-28,Havana,NaN,Cuba,73,190,1,Pitcher,1
3,669212,Eli Morgan,1996-05-13,2021-05-28,Rancho Palos Verdes,CA,USA,70,190,1,Pitcher,1
4,666201,Alek Manoah,1998-01-09,2021-05-27,Homestead,FL,USA,78,260,1,Pitcher,1


In [13]:
# df_engagement に df_players を LEFT JOIN
df_train = pd.merge(df_engagement, df_players, on=["playerId"], how="left")
print(df_train.shape)

(1003707, 21)


In [14]:
x_train = df_train[[
    "playerId",
    "dayofweek",
    "birthCity",
    "birthStateProvince",
    "birthCountry",
    "heightInches",
    "weight",
    "primaryPositionCode",
    "primaryPositionName",
    "playerForTestSetAndFuturePreds",
]]

y_train = df_train[[
    "target1",
    "target2",
    "target3",
    "target4",
]]

id_train = df_train[[
    "engagementMetricsDate",
    "playerId",
    "date_playerId",
    "date",
    "yearmonth",
    "playerForTestSetAndFuturePreds",
]]

print(x_train.shape, y_train.shape, id_train.shape)
x_train.head()

(1003707, 10) (1003707, 4) (1003707, 6)


,playerId,dayofweek,birthCity,birthStateProvince,birthCountry,heightInches,weight,primaryPositionCode,primaryPositionName,playerForTestSetAndFuturePreds
0,425794,2,Brunswick,GA,USA,79,230,1,Pitcher,1
1,571704,2,Albuquerque,NM,USA,75,210,1,Pitcher,0
2,506702,2,Maracaibo,NaN,Venezuela,70,235,2,Catcher,1
3,607231,2,Savannah,GA,USA,76,200,1,Pitcher,1
4,543193,2,Columbia,CA,USA,76,215,1,Pitcher,0


In [15]:
# lightgbmで扱うためcategory型に
for col in [
    "playerId",
    "dayofweek",
    "birthCity",
    "birthStateProvince",
    "birthCountry",
    "primaryPositionCode",
    "primaryPositionName",
]:
    x_train[col] = x_train[col].astype("category")

In [16]:
df_train.sort_values("date").tail()

,engagementMetricsDate,playerId,target1,target2,target3,target4,date_playerId,date,dayofweek,yearmonth,...,DOB,mlbDebutDate,birthCity,birthStateProvince,birthCountry,heightInches,weight,primaryPositionCode,primaryPositionName,playerForTestSetAndFuturePreds
1002325,2021-08-01,657011,0.0001,0.0583,0.0000,0.0551,20210801_657011,2021-07-31,5,2021-07,...,1996-02-02,2021-05-09,Columbia,MD,USA,69,205,7,Outfielder,1
1002324,2021-08-01,605335,0.0000,0.0137,0.0000,0.0184,20210801_605335,2021-07-31,5,2021-07,...,1992-12-13,2020-08-23,Alpharetta,GA,USA,76,190,1,Pitcher,0
1002323,2021-08-01,572308,0.0000,0.0069,0.0000,0.0110,20210801_572308,2021-07-31,5,2021-07,...,1990-03-01,2015-07-31,Don Gregorio de Nizao,NaN,Dominican Republic,71,210,1,Pitcher,0
1002289,2021-08-01,622668,0.0002,0.0926,0.0000,0.0294,20210801_622668,2021-07-31,5,2021-07,...,1995-05-30,2021-05-14,Santiago,NaN,Dominican Republic,69,179,4,Second Base,1
1003706,2021-08-01,623430,0.0000,0.0034,0.0000,0.0588,20210801_623430,2021-07-31,5,2021-07,...,1989-10-02,2015-09-01,Winston-Salem,NC,USA,69,185,1,Pitcher,0


# 8.3.4 バリデーション設計

## 説明変数として使って良いデータはなにか
- デジタルエンゲージメント（target1, target2, ...）は2021/7/31ぶんまで利用できる。それ以降の日付のぶんは利用できない
    - つまり説明変数として使う場合、8/1は前日より過去の値、8/2なら2日前よりも過去の値、・・・、8/31なら31日前より過去の値を利用できる
    - 31パターンのモデルを作成する手もあるが管理が大変なので、ベースラインでは1モデルとし、「先月のデジタルエンゲージメントなら利用可能」とする

## 古いデータは学習に使うべきか
- 著者の環境では期間を増やすほど精度が改善したとのことだが、ベースラインでは「直近1年」(=検証データ含め13ヶ月)とする

## 学習用データセットから検証データをどう作るか
- ベースラインでは検証データ期間を「2021/5」「2021/6」「2021/7」のパターンにわける。パターンごとに、直近12ヶ月を学習データとする

In [17]:
# 学習データと検証データの期間のリスト
list_cv_month = [
    [
        [
            "2020-05",
            "2020-06",
            "2020-07",
            "2020-08",
            "2020-09",
            "2020-10",
            "2020-11",
            "2020-12",
            "2021-01",
            "2021-02",
            "2021-03",
            "2021-04",
        ],
        [
            "2021-05",
        ],
    ],
    [
        [
            "2020-06",
            "2020-07",
            "2020-08",
            "2020-09",
            "2020-10",
            "2020-11",
            "2020-12",
            "2021-01",
            "2021-02",
            "2021-03",
            "2021-04",
            "2021-05",
        ],
        [
            "2021-06",
        ],
    ],
    [
        [
            "2020-07",
            "2020-08",
            "2020-09",
            "2020-10",
            "2020-11",
            "2020-12",
            "2021-01",
            "2021-02",
            "2021-03",
            "2021-04",
            "2021-05",
            "2021-06",
        ],
        [
            "2021-07",        
        ],
    ]
]

In [22]:
def train_lgb(
    input_x,
    input_y,
    input_id,
    params: dict,
    list_nfold=[0, 1, 2],
    mode_train="train",
):
    # 推論値を格納する変数
    df_valid_pred = pd.DataFrame()
    # 評価値を入れる変数
    metrics = []
    # 重要度を格納する変数
    df_imp = pd.DataFrame()
    
    # corss validation 用のインデクス作成
    cv = []
    for month_tr, month_va in list_cv_month:
        _cv = [
            input_id.index[input_id["yearmonth"].isin(month_tr)],
            input_id.index[input_id["yearmonth"].isin(month_va) & (input_id["playerForTestSetAndFuturePreds"] == 1)],
        ]
        cv.append(_cv)

    # fold * target 毎にモデル学習
    for nfold in list_nfold:
        for i, target in enumerate(["target1", "target2", "target3", "target4"]):
            print("-"*20, target, ", fold:", nfold, "-"*20)
            
            idx_tr, idx_va = cv[nfold][0], cv[nfold][1]
            x_tr, y_tr, id_tr = input_x.loc[idx_tr, :], input_y.loc[idx_tr, target], input_id.loc[idx_tr, :]
            x_va, y_va, id_va = input_x.loc[idx_va, :], input_y.loc[idx_va, target], input_id.loc[idx_va, :]
            print(x_tr.shape, y_tr.shape, id_tr.shape)
            print(x_va.shape, y_va.shape, id_va.shape)
            
            # 保存するモデルのファイル名
            filepath = "model_lgb_{}_fold{}.h5".format(target, nfold)
            
            if mode_train == "train":
                print("training start.")
                model = lgb.LGBMRegressor(**params)
                model.fit(
                    x_tr,
                    y_tr,
                    eval_set=[(x_tr, y_tr), (x_va, y_va)],
                    callbacks=[lgb.early_stopping(stopping_rounds=50),],
                )
                with open(filepath, "wb") as f:
                    pickle.dump(model, f, protocol=4)
            else:
                print("model load.")
                with open(filepath, "rb") as f:
                    model = pickle.load(f)
                print("Done.")
            
            # validの推論値取得
            y_va_pred = model.predict(x_va)
            tmp_pred = pd.concat(
                [
                    id_va,
                    pd.DataFrame({
                        "target": target,
                        "nfold": nfold,
                        "true": y_va,
                        "pred": y_va_pred,
                    })
                ],
                axis=1,
            )
            df_valid_pred = pd.concat([df_valid_pred, tmp_pred], axis=0, ignore_index=True)
            
            # 評価値の算出
            metric_va = mean_absolute_error(y_va, y_va_pred)
            metrics.append([target, nfold, metric_va])
            
            # 重要度
            tmp_imp = pd.DataFrame({
                "col": x_tr.columns,
                "imp": model.feature_importances_,
                "target": target,
                "nfold": nfold,
            })
            df_imp = pd.concat(
                [
                    df_imp,
                    tmp_imp,
                ],
                axis=0,
                ignore_index=True
            )
            
    print("-"*10, "result", "-"*10)
    # 評価値
    df_metrics = pd.DataFrame(metrics, columns=["target", "nfold", "mae"])
    print("MCMAE: {:.4f}".format(df_metrics["mae"].mean()))
    
    # validの推論値
    df_valid_pred_all = pd.pivot_table(
        df_valid_pred,
        # プレイヤー * 日付 ごとに
        index=[
            "engagementMetricsDate",
            "playerId",
            "date_playerId",
            "date",
            "yearmonth",
            "playerForTestSetAndFuturePreds",
        ],
        # target * fold ごとの
        columns=[
            "target",
            "nfold",
        ],
        # 真の値と予測値を算出
        values=[
            "true",
            "pred"
        ],
        # なぜ sum を使うのか?
        aggfunc=np.sum,
    )
    df_valid_pred_all.columns = [
        "{}_fold{}_{}".format(j, k, i) for i, j, k in df_valid_pred_all.columns
    ]
    df_valid_pred_all = df_valid_pred_all.reset_index(drop=False)
    
    return df_valid_pred_all, df_metrics, df_imp

In [23]:
# モデル学習
params = {
    "boosting_type": "gbdt",
    "objective": "regression_l1",
    "metric": "mean_absolute_error",
    "learning_rate": 0.05,
    "num_leaves": 32,
    "subsample": 0.7,
    "subsample_freq": 1,
    "feature_fraction": 0.8,
    "min_data_in_leaf": 50,
    "min_sum_hessian_in_leaf": 50,
    "n_estimators": 1000,
    "random_state": 123,
    "importance_type": "gain",
}

df_valid_pred, df_metrics, df_imp = train_lgb(
    x_train,
    y_train,
    id_train,
    params,
)

-------------------- target1 , fold: 0 --------------------
(752265, 10) (752265,) (752265, 6)
(36797, 10) (36797,) (36797, 6)
training start.
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=50, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=50, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: mi

In [29]:
print("MCMAE: {:.4f}".format(df_metrics["mae"].mean()))
display(
    pd.pivot_table(
        df_metrics,
        index="nfold",
        columns="target",
        values="mae",
        aggfunc=np.mean,
        margins=True,
    )
)

MCMAE: 1.3504


target,target1,target2,target3,target4,All
nfold,,,,,
0,1.2977,2.4447,0.8780,1.2451,1.4664
1,1.1953,2.1539,0.8317,1.6406,1.4554
2,1.1133,1.7903,0.7606,0.8542,1.1296
All,1.2021,2.1297,0.8234,1.2466,1.3504


In [30]:
df_imp.groupby(["col"])["imp"].agg(["mean", "std"]).sort_values("mean", ascending=False)

,mean,std
col,,
playerId,4976980.5395,6102371.1341
playerForTestSetAndFuturePreds,1115074.5464,1091298.8384
birthCity,741457.7171,1058010.6632
primaryPositionCode,110792.2785,167555.0234
dayofweek,78879.0470,140714.7657
primaryPositionName,33697.4310,38239.7619
weight,20633.3897,31551.4583
heightInches,19410.1825,34119.9997
birthStateProvince,7714.1863,13322.8164


# 8.3.6 モデル推論